In [3]:
!pip install opencv-python==4.11.0.86
!pip install ultralytics

  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp310-cp310-win_amd64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ---------------------

In [5]:
from pathlib import Path
import cv2
from ultralytics import YOLO

# Загрузка модели
# model = YOLO('runs/segment/train7/weights/best.pt')
# model = YOLO('yolo11m.pt')
# model = YOLO('runs/detect/train22/weights/best.pt')
# model = YOLO('runs/detect/train13/weights/best.pt')
# model = YOLO('runs/detect/train27/weights/best.pt') # 200
# model = YOLO('runs/detect/train29/weights/best.pt') # 300
model = YOLO('runs/detect/train4/weights/best.pt') # 400

# Загрузка изображения
image_path = str(Path.cwd() / 'images' / '3' / '1.jpg')
img = cv2.imread(image_path)


# Получение предсказаний
results = model(img, verbose=False, conf=0.4)

# Настройка визуализации
annotated_image = results[0].plot(
    masks=False,    # Отключаем маски
    line_width=1,   # Толщина обводки
    labels=True,     # Включаем названия классов
    boxes=True
)

# Сохранение результата
output_path = '1.jpg'
cv2.imwrite(output_path, annotated_image)

True

In [7]:
import numpy as np

def point_in_polygon(point, polygon):
    """
    Проверяет, находится ли точка внутри полигона.
    :param point: (x, y) - координаты точки
    :param polygon: [(x1, y1), (x2, y2), ...] - вершины полигона
    :return: True если точка внутри, иначе False
    """
    x, y = point
    n = len(polygon)
    inside = False
    
    p1x, p1y = polygon[0]
    for i in range(1, n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    
    return inside

polygon_right_center = [(0, 278), (146, 229), (233, 436), (0, 502)]
polygon_right_right = [(255, 179), (552, 429), (787, 300), (356, 151)]
polygon_right_top = [(0, 246), (226, 171), (246, 203), (0, 287)]

polygon_bottom_bottom = [(260, 173), (1277, 339), (1277, 440), (179, 194)]
polygon_bottom_left = [(730, 246), (1003, 235), (798, 209), (541, 212)] # переделать
polygon_bottom_top = [(1246, 276), (837, 210), (846, 189), (1250, 256)]

In [8]:
import cv2
import numpy as np
from pathlib import Path

# Загрузка изображения (замените на ваш путь)
image_path = str(Path.cwd() / 'images' / '3'/ '1.jpg')
image = cv2.imread(image_path)


# Ваши полигоны
polygon_right_center = [(0, 288), (146, 239), (233, 436), (0, 502)]
polygon_right_right = [(255, 179), (552, 429), (787, 300), (356, 151)]
polygon_right_top = [(0, 246), (226, 171), (246, 203), (0, 287)]

# Конвертируем точки в numpy-массив (требуется для OpenCV)
polygons = [
    np.array(polygon_right_center, dtype=np.int32),
    np.array(polygon_right_right, dtype=np.int32),
    np.array(polygon_right_top, dtype=np.int32)
]

# Цвета для каждого полигона (BGR)
colors = [
    (0, 255, 0),   # Зеленый
    (0, 0, 255),   # Красный
    (255, 0, 0)    # Синий
]

# Отрисовка полигонов
for i, polygon in enumerate(polygons):
    # Заливка полигона (с прозрачностью)
    overlay = image.copy()
    cv2.fillPoly(overlay, [polygon], colors[i])
    alpha = 0.3  # Прозрачность
    image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
    
    # Контур полигона
    cv2.polylines(image, [polygon], isClosed=True, color=colors[i], thickness=2)

# Сохранение результата
cv2.imwrite('image_with_polygons.jpg', image)

True

In [8]:
counter_polygon_right_center = 0
counter_polygon_right_right = 0
counter_polygon_right_top = 0

counter_polygon_bottom_bottom = 0
counter_polygon_bottom_left = 0
counter_polygon_bottom_top = 0

for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # получаем углы
        center_x = (x1 + x2) // 2  # центр по X
        center_y = (y1 + y2) // 2  # центр по Y
        center = (center_x, center_y)
        
        counter_polygon_right_center += 1 if point_in_polygon(center, polygon_right_center) else 0
        counter_polygon_right_right +=1 if point_in_polygon(center, polygon_right_right) else 0
        counter_polygon_right_top +=1 if point_in_polygon(center, polygon_right_top) else 0
        
        counter_polygon_bottom_bottom += 1 if point_in_polygon(center, polygon_bottom_bottom) else 0
        counter_polygon_bottom_left += 1 if point_in_polygon(center, polygon_bottom_left) else 0
        counter_polygon_bottom_top += 1 if point_in_polygon(center, polygon_bottom_top) else 0

        
print(counter_polygon_right_center, counter_polygon_right_right, counter_polygon_right_top,
     counter_polygon_bottom_bottom, counter_polygon_bottom_left, counter_polygon_bottom_top)

15 6 3 6 2 0


In [1]:
import torch

# Проверка версии PyTorch
print(f"PyTorch version: {torch.__version__}")

# Проверка доступности CUDA (если есть GPU)
print(f"CUDA available: {torch.cuda.is_available()}")

# Версия CUDA, с которой собран PyTorch
print(f"CUDA version (PyTorch build): {torch.version.cuda}")

# Количество доступных GPU
print(f"Number of GPUs: {torch.cuda.device_count()}")

# Название текущего GPU (если есть)
if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected")

PyTorch version: 2.7.0+cu126
CUDA available: True
CUDA version (PyTorch build): 12.6
Number of GPUs: 1
Current GPU: NVIDIA GeForce RTX 2080 Ti


In [1]:
import torch
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11m-seg.pt")  # build a new model from YAML
# Train the model
results = model.train(
    data="dataset/data.yaml", 
    epochs=150, 
    imgsz=640, 
    batch=8, 
    model="yolo11m-seg.pt",
    amp=False
)

New https://pypi.org/project/ultralytics/8.3.138 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.135  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11264MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, opset=None, op

train: Scanning E:\PYproject\honnypye\YOLO\dataset\train\labels.cache... 85 images, 1 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 1716.4473.5 MB/s, size: 1936.5 KB)


val: Scanning E:\PYproject\honnypye\YOLO\dataset\validation\labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]


Plotting labels to runs\segment\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 115 weight(decay=0.0), 126 weight(decay=0.0005), 125 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train7
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...


150 epochs completed in 4.485 hours.
Optimizer stripped from runs\segment\train7\weights\last.pt, 45.2MB
Optimizer stripped from runs\segment\train7\weights\best.pt, 45.2MB

Validating runs\segment\train7\weights\best.pt...
Ultralytics 8.3.135  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11264MiB)
YOLO11m-seg summary (fused): 138 layers, 22,336,083 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:0

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:0


                   all         15       1289        0.9      0.704      0.807      0.609      0.855      0.667      0.756      0.513
Speed: 1.0ms preprocess, 31.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\segment\train7


In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import torch
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11m.pt")
# Train the model
results = model.train(
    data="dataset/data.yaml",
    amp=True,
    epochs=200, 
    imgsz=736,
    device=0,
    batch=12, 
    model="yolo11m.pt",
    augment=True,
    mosaic=1.0,
    mixup=0.3,
    copy_paste=0.2,
    degrees=15,
    scale=0.7,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4
)

New https://pypi.org/project/ultralytics/8.3.159 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.135  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11264MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=736, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.3, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=Fa

train: Scanning E:\PYproject\honnypye\YOLO\dataset\train\labels... 304 images, 18 backgrounds, 0 corrupt: 100%|████████

train: New cache created: E:\PYproject\honnypye\YOLO\dataset\train\labels.cache


val: Fast image access  (ping: 0.10.0 ms, read: 1723.6417.7 MB/s, size: 1502.6 KB)


val: Scanning E:\PYproject\honnypye\YOLO\dataset\val\labels... 68 images, 3 backgrounds, 0 corrupt: 100%|██████████| 71

...

    200/200      9.51G     0.6885     0.3806     0.9329        470        736: 100%|██████████| 27/27 [00:08<00:00,  3.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<0

                   all         71       4323      0.947      0.844      0.912      0.741



200 epochs completed in 2.592 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 40.5MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.135  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11264MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<0


                   all         71       4323      0.932      0.827      0.907      0.741
Speed: 0.6ms preprocess, 32.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\train4
